# Launch pytorch traing jobs for conditional GAN
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [14]:
import os
import glob,time
import subprocess as sp
import numpy as np

In [15]:
curr_dir=os.getcwd()
print(curr_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results


In [16]:
code_dirs={'128':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code',
          '512':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code' }

config={'128': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/config_128.yaml',
        '512': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/config_256.yaml'}

In [17]:
### Default dictionary
img_size=128
dict_pars={'epochs':20, 'batch_size':64, 'nodes':1,'seed':36723705, 'spec_loss': '','ip_fldr':'-',
           'model':2,
           'job_name':'cgan_predict_0.8',
#            'ip_fldr':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201014_073327/',
           'spec_loss':'--specloss',
            'deterministic':'--deterministic',
           'save_steps_list':'32750',
           'sigma_list':'0.5 0.65 1.1',
           'lr':0.0002,
           'lambda1':1.0,
           'procs':8, 'time':'4:00:00','mode':'fresh'}

dict_pars['code_dir']=code_dirs[str(img_size)]
dict_pars['config_file']=config[str(img_size)]
dict_pars['job_name']=dict_pars['job_name']+'_m'+str(dict_pars['model'])
print(dict_pars)

{'epochs': 20, 'batch_size': 64, 'nodes': 1, 'seed': 36723705, 'spec_loss': '--specloss', 'ip_fldr': '-', 'model': 2, 'job_name': 'cgan_predict_0.8_m2', 'deterministic': '--deterministic', 'save_steps_list': '32750', 'sigma_list': '0.5 0.65 1.1', 'lr': 0.0002, 'lambda1': 1.0, 'procs': 8, 'time': '4:00:00', 'mode': 'fresh', 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code', 'config_file': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/config_128.yaml'}


In [18]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node={procs}
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME
### Initial setup

conda activate v3
### module load pytorch-v1.4.0-gpu
##module load esslurm

code_dir={code_dir}
config_file={config_file}
python $code_dir/main.py --seed {seed} {spec_loss} {deterministic}  -l {save_steps_list} -sl {sigma_list} --config $config_file --mode {mode} --model {model} --ngpu {procs} --learn_rate {lr} --batch_size {batch_size} --epochs {epochs} -ld1 {lambda1} -ip {ip_fldr} -rs {job_name}

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [19]:
# bash_strg

In [20]:
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'

In [21]:
fname='batch_train_cgan.sh'
filename=staging_loc+fname
with open (filename,'w') as f:
    f.write(bash_strg)

In [22]:
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_train_cgan.sh


In [23]:
### Move to staging locations in project space:
os.chdir(staging_loc)

## Submit job to cori GPU

In [24]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time=4:00:00
#SBATCH --job-name=cgan_predict_0.8_m2

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME
### Initial setup

conda activate v3
### module load pytorch-v1.4.0-gpu
##module load esslurm

code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code
config_file=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/config_128.yaml
python $code_dir/main.py --seed 36723705 --specloss --deterministic  -l 32750 -sl 0.5 0.65 1.1 --config $config_file --mode fresh --model 2 --ngpu 8 --learn_rate 0.0002 --batch_size 64 --epochs 20 -ld1 1.0 -ip - -rs cgan_predict_0.8_m2

echo "--end date" `date` `date +%s

In [25]:
%%bash -s "$filename" ## Use python variable in bash
module load cgpu
sbatch $1

Submitted batch job 1456242
